# Outline
- ## Introduction to Julia
- ## HPC in Julia
- ## Julia in Data Science (with R and Python and Lale)
- ## Use-case: Covid Data Explorations, ML Pipeline Optimization

# What were the languages designed for?
- ### Fortran: for HPC and technical computing (IBM, 1950s, 71 years ago)
- ### LISP: for symbolic programming (MIT, 1958)
- ### C: to write Unix OS (Bell labs,1970s, 50 years ago)
- ### IDL: for data analysis (Univ. of Colorado, 1970s)
- ### S or R: for statistical analysis (Bell labs, 1976, 45 years ago)
- ### Matlab: for numerical computing (University of New Mexico, 1984)
- ### C++: for system programming, embedded, large systems (Bell labs, 1978/1985)
- ### PERL: for generating reports (Unisys, 1987)
- ### Ruby: general purpose, object-oriented scripting language (University of Tsukuba, 1990, 30 years ago)
- ### Java: cross-platform, hand-held devices, interactive television (Sun Microsystems, 1991, 30 years ago)
- ### Python: general-purpose programming (Centrum Wiskunde & Informatica, 1991, 30 years ago)
- ### Lua: for embedded systems, extend applications, customisation (Pontifical Catholic University, 1993)
- ### Scala: as a better Java (EPFL, 2004)
- ### Rust: designed for performance and safety, especially safe concurrency (Mozilla, 2012, 9 years ago)
- ### Julia: optimised for HPC, scientific computing, general purpose programming (MIT, 2012, 9 years ago)¶

# Programming

- ### INPUT => PROCESS => OUTPUT

- ### translating high-level language (for humans) into low-level language (for Machines)

# Programming Language Types
- ### Interpreter (R, Python, Perl) - highly interactive, real time translation
     - #### REPL: Read, Eval, Print, Loop (line-by-line)
- ### Compiler (Fotran, C, C++, Rust) - batch/offline translation
- ### Bytecode Interpreter (Java, Scala) - batch/offline translation
- ### Just-in-time Compiler (Julia, Lua) - highly interactive, just-in-time batch translation
    - #### REPL: Read, Eval, Print, Loop (batch and cached)

# Easy enough function to understand: $ sum(x) = \sum_{i=1}^{n} x_{i} $

In [1]:
x = rand(10^8)

100000000-element Vector{Float64}:
 0.6248736382530575
 0.5009191391326739
 0.9032790380829836
 0.45422432675258007
 0.8146554106489878
 0.03794501481471335
 0.11108903188910046
 0.5952175064530325
 0.27080496740684
 0.09089249278069245
 0.876082602652229
 0.18081478697335363
 0.959994321936569
 ⋮
 0.050837588964610436
 0.9948512978699344
 0.34665704003282727
 0.8395833159288137
 0.9337302318403626
 0.1297568738447783
 0.10899095941929304
 0.48856869639443357
 0.34044038023828027
 0.23715086701329402
 0.2501391323263633
 0.6039407870986666

In [2]:
function jsum(v)
    s = 0
    for i in v
        s = s + i
    end
    return s
end

jsum (generic function with 1 method)

In [12]:
@time jsum(x)

  0.312313 seconds (1 allocation: 16 bytes)


5.0004465303068236e7

In [4]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""
const Clib = tempname()   # make a temporary file
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
@time c_sum(x)

  0.283869 seconds (2.23 k allocations: 127.930 KiB, 52.80% compilation time)


5.0004465303068236e7

In [6]:
using RCall
R"""
rsum <- function(v) {
  s = 0
  for(i in v){
    s = s + i
  }
  return(s)
}
""";

In [7]:
@time R"rsum($x)" |> rcopy

  2.352640 seconds (375.45 k allocations: 23.091 MiB, 8.08% compilation time)


5.0004465303068236e7

In [8]:
using PyCall
py"""
def psum(v):
   s=0
   for i in v:
     s = s + i
   return s
"""

In [9]:
@time py"psum"(x)

 10.808344 seconds (730.26 k allocations: 43.064 MiB, 2.96% compilation time)


5.0004465303068236e7

In [13]:
@code_native jsum(x)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[2]:1 within `jsum'
	pushq	%rax
; │ @ In[2]:3 within `jsum'
; │┌ @ array.jl:777 within `iterate' @ array.jl:777
; ││┌ @ array.jl:197 within `length'
	movq	8(%rsi), %r9
; ││└
; ││┌ @ int.jl:448 within `<' @ int.jl:441
	testq	%r9, %r9
; ││└
	je	L100
; ││┌ @ array.jl:801 within `getindex'
	movq	(%rsi), %r8
; │└└
	decq	%r9
	movb	$1, %cl
	xorl	%edx, %edx
; │┌ @ array.jl:777 within `iterate' @ array.jl:777
	xorl	%esi, %esi
	xorl	%eax, %eax
	nopl	(%rax,%rax)
; │└
; │ @ In[2] within `jsum'
L32:
	vmovsd	(%r8,%rdx,8), %xmm0             ## xmm0 = mem[0],zero
; │ @ In[2]:4 within `jsum'
	testb	$1, %cl
	je	L64
	testb	$1, %al
	jne	L113
; │┌ @ promotion.jl:321 within `+'
; ││┌ @ promotion.jl:292 within `promote'
; │││┌ @ promotion.jl:269 within `_promote'
; ││││┌ @ number.jl:7 within `convert'
; │││││┌ @ float.jl:94 within `Float64'
	vcvtsi2sd	%rsi, %xmm2, %xmm1
	jmp	L69
	nopw	%cs:(%rax,%rax)
; │└└└└└
; │┌ @ float.jl:326 within `+'
L64:
	vmovq

## Brief History of Julia
- ### 2009: initial work started by Jeff Bezanson, Stefan Karpinski, Viral B. Shah, and Prof. Alan Edelman
- ### 2012: Julia public announcement
- ### 2014-2017: versions 0.3-0.7 (yearly release of major features towards 1.0)
- ### 2015: **Julia Computing** was founded by Julia creators for production support
- ### 2017: **IBM PowerPC port of Julia sponsored by IBM**
- ### 2018: 1.0 Stable (LTS version) 
- ### 2020: latest version is Julia 1.6.2, with Julia 1.0.5 as LTS
- ### 2020: 13M downloads, used by 1500+ Universities
- ### 2021: 29M+ downloads, 203k+ github stars, 5.9k packages
     - ### runs in Windows, MacOS, Linux, FreeBSD OS
     - ### runs in X86, ARM, PowerPC hardwares


# Benchmarks
- ### https://julialang.org/benchmarks/

# Links
- ### **Julia Language**: https://julialang.org/
- ### **Julia Download**: https://julialang.org/downloads/
- ### **Julia Computing**: https://juliacomputing.com/
- ### **Julia in Industry**: https://juliacomputing.com/case-studies/
- ### **Julia Community Help Channel**: https://discourse.julialang.org/
- ### **Julia Youtube Channel**: https://www.youtube.com/user/julialanguage

# Julia Conference 2021 (July 28 - 30)
- ### **Julia Conference 2021**: https://juliacon.org/2021/
    - free registration


# My Github Repo
- ### Main Link: https://github.com/ppalmes
- ### Online Workshop: https://github.com/ppalmes/JuliaWorkshop